In [1]:
import json
import pandas as pd
import os
# uuid
from django.utils.crypto import get_random_string


convert
1. json -- index
    corres (uuid) - subtopic
    subtopic - topic

    node
    {
    "labels": ["correspondence"],
    "id": uuid
    },
    {
    "labels": ["subtopic"],
    "id": get_random_string(8)
    },
    {
    "labels": ["topic"],
    "id": get_random_string(8)
    },

    link
    {
    "type": "topic_subtopic",
    "id": 147,
    "source": 21,
    "target": 344,
    "value": 1
    },
    {
    "type": "corres_subtopic",
    "id": 147,
    "source": 21,
    "target": 344,
    "value": 1
    },
    

2. csv -- people

config link--match property

In [9]:

#********************************** index

with open(os.path.expanduser('data_prep/r_e_corres_theme_group.json')) as f:
    corres_list = json.load(f)

node_list, link_list = [], []
node_corr_list, node_top_list, node_subtop_list = [], [], []
link_corr_top_list, link_corr_subtop_list, link_topic_subtop_list = [], [], []

# print
for corr in corres_list:
    node_corr_dict={
        "labels": ["correspondence"],
        "id": corr['unique_id'], # can hardly be replaced by the name
        "properties": {
            "name": corr['from']+" TO "+corr['to']+" "+str(corr['date']).replace("'", '').replace("]", "").replace("[", ""),
            "value": 0,
            "count_mentioned": 1,

            "from": corr['from'],
            "to": corr['to'],
            "date": corr['date'],
            "grouped_themes_count": corr["label"]["index_shapiro"]["grouped_themes_count"],
            "grouped_themes": str(corr["label"]["index_shapiro"]["grouped_themes"]).replace("'", '"').replace("], ", "<br>").replace("]", "").replace("[", "").replace("{", "").replace("}", ""),
            "grouped_themes_count_detailed": str(corr["label"]["index_shapiro"]["grouped_themes_count_detailed"]).replace(", '", '<br/>').replace("': ", ': ').replace("}", '').replace("{'", ''),

            "location": corr["location"],
            "translated_text": corr["content"]["text"]["translated_text"],
            }
        }
    node_corr_list.append(node_corr_dict)

    for top in corr["label"]["index_shapiro"]["grouped_themes"].keys():
        # increase value of node-correspondence
        node_corr_dict["properties"]['value'] += 1

        # record information
        node_top_dict={
            "labels": ["topic"],
            "id": get_random_string(8),
            "properties": {
                "name": top,
                "count_subtopic": 0,
                "count_mentioned": 1,
                "subtopics": corr["label"]["index_shapiro"]["grouped_themes"][top]
            }
            }
        link_corr_top_dict={
            "type": "corres_topic",
            "id": get_random_string(8),
            "source": node_corr_dict["properties"]['name'],
            "target": node_top_dict["properties"]['name'],
            "properties": {
                "value": 1
            }
            }
        node_top_list.append(node_top_dict)
        link_corr_top_list.append(link_corr_top_dict)

        for subtop in corr["label"]["index_shapiro"]["grouped_themes"][top]:

            # record information
            node_subtop_dict={
                "labels": ["subtopic"],
                "id": get_random_string(8),
                "properties": {
                    "name": subtop,
                    "value": 0,
                    "count_mentioned": 1,
                    "parent_topic": top
                }
                }
            link_corr_subtop_dict={
                "type": "corres_subtopic",
                "id": get_random_string(8),
                "source": node_corr_dict["properties"]['name'],
                "target": node_subtop_dict["properties"]['name'],
                "properties": {
                    "value": 1
                }
                }
            link_topic_subtop_dict={
                "type": "topic_subtopic",
                "id": get_random_string(8),
                "source": node_top_dict["properties"]['name'],
                "target": node_subtop_dict["properties"]['name'],
                "properties": {
                    "value": 1
                }
                }
            node_subtop_list.append(node_subtop_dict)
            link_corr_subtop_list.append(link_corr_subtop_dict)
            link_topic_subtop_list.append(link_topic_subtop_dict)

# remove the duplicates from all lists of dicts
def remove_duplicates_node(list_of_dicts):
    # dulicated if node["properties"]['name'] are the same
    unique_list, unique_name_list = [], []
    for node in list_of_dicts:
        if node["properties"]['name'] not in unique_name_list:
            unique_name_list.append(node["properties"]['name'])
            unique_list.append(node)
        # increase count_mentioned if duplicated
        else:
            for unique_node in unique_list:
                if node["properties"]['name'] == unique_node["properties"]['name']:
                    # increase count_mentioned
                    unique_node["properties"]['count_mentioned'] += 1
                    # append to the list of subs if ["properties"]["subtopic"] exists
                    if "subtopics" in node["properties"].keys():
                        unique_node["properties"]["subtopics"] += node["properties"]["subtopics"]
    
    # if ["properties"]["subtopic"] exists, remove duplicates and change the format
    # str(["properties"]["subtopic"]).replace("', '","<br/>").replace("'", "").replace("]", "").replace("[", "")
    for node in unique_list:
        if "subtopics" in node["properties"].keys():
            node["properties"]["subtopics"] = list(set(node["properties"]["subtopics"]))
            # set count_subtopic
            node["properties"]["count_subtopic"] = len(node["properties"]["subtopics"])
            # change and clean the format
            node["properties"]["subtopics"] = str(node["properties"]["subtopics"]).replace("', '","<br/>").replace("'", "").replace("]", "").replace("[", "")
    return unique_list

node_corr_list = remove_duplicates_node(node_corr_list)
node_top_list = remove_duplicates_node(node_top_list)
node_subtop_list = remove_duplicates_node(node_subtop_list)

def remove_duplicates_link(list_of_dicts):
    # dulicated if source and target are the same
    # add up value if duplicated
    unique_list, unique_name_list = [], []
    for link in list_of_dicts:
        if link['source']+" "+link['target'] not in unique_name_list:
            unique_name_list.append(link['source']+" "+link['target'])
            unique_list.append(link)
        else:
            for unique_link in unique_list:
                if link['source'] == unique_link['source'] and link['target'] == unique_link['target']:
                    unique_link["properties"]['value'] += 1
        
    # replace names of source and target with ids
    node_all_list = node_corr_list + node_top_list + node_subtop_list
    for link in unique_list:
        for node in node_all_list:
            if link['source'] == node["properties"]['name']:
                link['source'] = node['id']
                source_name = node["properties"]['name']
            if link['target'] == node["properties"]['name']:
                link['target'] = node['id']
                target_name = node["properties"]['name']
        link["properties"]['name'] = source_name + " -> " + target_name

    return unique_list

link_corr_top_list = remove_duplicates_link(link_corr_top_list)
link_corr_subtop_list = remove_duplicates_link(link_corr_subtop_list)
link_topic_subtop_list = remove_duplicates_link(link_topic_subtop_list)


# how to deal with self to self--corres directly mention the topic not the subtopic
# subtopic include the topic name


In [10]:
# combine
# topic - (subtopic) - corres
node_list = node_corr_list + node_top_list
link_list = link_corr_top_list
node_link_dict = {"nodes": node_list, "links": link_list}
# josn dump
with open(os.path.expanduser('js/07-16-2023/graph_corr_topic.js'), 'w') as f:
    # write to json file "var graph_corr_topic ="
    f.write("var graph_corr_topic = ")
    json.dump(node_link_dict, f, indent=4)
    

In [11]:
#********************************** graph_phil_pv
# node: people, connections
# link: the connection, with types, directed?
# people - connection - people

# read json file /Users/junyitao/Documentation/research/SW/shapiro_correspondence/correspondence_index/data_prep/people.json
with open(os.path.expanduser('data_prep/people.json')) as f:
    people_list = json.load(f)


node_list, link_list = [], []
node_people_list, node_conn_list = [], []
link_people_conn_list = []

for people in people_list:
    node_people_dict = { # no depulicates
        "labels": ["people"],
        "id": get_random_string(8),
        "properties":{
            "name": people['name'],
            "sameAs": people['sameAs'],
            "roles": people['roles'],
            "connections": people['connections'],
            "value": 0
        }
    }
    node_people_list.append(node_people_dict)

    for conn in people['connections']:
        # increase value of node-people
        node_people_dict["properties"]['value'] += 1

        # record information
        node_conn_dict = {
            "labels": ["connection"],
            "type": conn["type"],
            "id": conn["unique_id"],
            "properties":{
                "name": conn["from"]+" TO "+conn["to"]+" "+str(conn["date"]).replace("[", '').replace("]", ''),
                "value": 0,
                "people": people['name'],
                "source": conn["source"],
                "from": conn["from"],
                "to": conn["to"],
                "date": conn["date"],
                "location": conn["location"],
                "translated_text": conn["content"]["text"]["translated_text"]
            }
        }
        link_people_conn_dict = {
            "type": "people_connection",
            "id": get_random_string(8),
            # here we use id of people and connections
            # they are supposed to be depulicated twice
            "source": node_people_dict["id"],
            "target": node_conn_dict["id"],
            "properties": {
                "value": 0
            }
        }
        node_conn_list.append(node_conn_dict)
        link_people_conn_list.append(link_people_conn_dict)

# remove duplicates of node_conn_list, with id
def remove_duplicates_node_conn(list_of_dicts):
    # dulicated if node["properties"]['name'] are the same
    unique_list, unique_name_list = [], []
    for node in list_of_dicts:
        if node["id"] not in unique_name_list:
            unique_name_list.append(node["id"])
            unique_list.append(node)
    
    return unique_list

node_conn_list = remove_duplicates_node_conn(node_conn_list)

node_conn_dict

{'labels': ['connection'],
 'type': 'correspondence',
 'id': '846e3c2b-aef2-4cc3-8dd3-f15bca338e17',
 'properties': {'name': "Samson Johnson TO Sorbiere, Samuel '08/27/1644'",
  'value': 0,
  'people': 'Samson Johnson',
  'source': {'projectvox_team': ['Alexandrescu 17'],
   'electronic_enlightenment': ''},
  'from': 'Samson Johnson',
  'to': 'Sorbiere, Samuel',
  'date': ['08/27/1644'],
  'location': [],
  'translated_text': ''}}

In [12]:
# combine
# people - connection - people
node_list = node_people_list + node_conn_list
link_list = link_people_conn_list
node_link_dict = {"nodes": node_list, "links": link_list}
# josn dump
with open(os.path.expanduser('js/07-16-2023/graph_phil_pv.js'), 'w') as f:
    # write to json file "var graph_phil_pv="
    f.write("var graph_phil_pv = ")
    json.dump(node_link_dict, f, indent=4)

## Exprimental part (unused)

In [ ]:
#********************************** graph_phil_other

# read from csv

# medium nodes are prefered to links (with label) here because straight lines overlap with each other

node_list, link_list = [], []
node_people_list, node_conn_list = [], []
link_people_conn_list = []

for people in people_list:
    node_people_dict = { # no depulicates
        "labels": ["people"],
        "id": get_random_string(8),
        "properties":{
            "name": people['name'],
            "sameAs": people['sameAs'],
            "roles": people['roles'],
            "connections": people['connections'],
            "value": 0
        }
    }
    node_people_list.append(node_people_dict)

    for conn in people['connections']:
        # increase value of node-people
        node_people_dict["properties"]['value'] += 1

        # record information
        node_conn_dict = {
            "labels": ["connection"],
            "type": conn["type"],
            "id": conn["unique_id"],
            "properties":{
                "name": conn["from"]+" TO "+conn["to"]+" "+str(conn["date"]).replace("[", '').replace("]", ''),
                "value": 0,
                "people": people['name'],
                "source": conn["source"],
                "from": conn["from"],
                "to": conn["to"],
                "date": conn["date"],
                "location": conn["location"],
                "translated_text": conn["content"]["text"]["translated_text"]
            }
        }
        link_people_conn_dict = {
            "type": "people_connection",
            "id": get_random_string(8),
            # here we use id of people and connections
            # they are supposed to be depulicated twice
            "source": node_people_dict["id"],
            "target": node_conn_dict["id"],
            "properties": {
                "value": 0
            }
        }
        node_conn_list.append(node_conn_dict)
        link_people_conn_list.append(link_people_conn_dict)

# remove duplicates of node_conn_list, with id
def remove_duplicates_node_conn(list_of_dicts):
    # dulicated if node["properties"]['name'] are the same
    unique_list, unique_name_list = [], []
    for node in list_of_dicts:
        if node["id"] not in unique_name_list:
            unique_name_list.append(node["id"])
            unique_list.append(node)
    
    return unique_list

node_conn_list = remove_duplicates_node_conn(node_conn_list)

In [ ]:
graph_phil_other

In [38]:
link_corr_top_list

[{'type': 'corres_topic',
  'id': '30vJQbcR',
  'source': '4beed4b9-b5ad-4d85-af2e-6fa2dcfbed7b',
  'target': 'FSHmfGTa',
  'value': 1,
  'name': "Princess Elisabeth TO Descartes, René ['05/06/1643'] -> voluntary action"},
 {'type': 'corres_topic',
  'id': 'TpXCAmIF',
  'source': '4beed4b9-b5ad-4d85-af2e-6fa2dcfbed7b',
  'target': 'w1oqbthx',
  'value': 1,
  'name': "Princess Elisabeth TO Descartes, René ['05/06/1643'] -> souls"},
 {'type': 'corres_topic',
  'id': 'bLxCgMg0',
  'source': '4beed4b9-b5ad-4d85-af2e-6fa2dcfbed7b',
  'target': 'jpZRDCpS',
  'value': 1,
  'name': "Princess Elisabeth TO Descartes, René ['05/06/1643'] -> action(s), human"},
 {'type': 'corres_topic',
  'id': 'JW05O55o',
  'source': '4beed4b9-b5ad-4d85-af2e-6fa2dcfbed7b',
  'target': 'wemchvia',
  'value': 1,
  'name': "Princess Elisabeth TO Descartes, René ['05/06/1643'] -> substance(s)"},
 {'type': 'corres_topic',
  'id': 'ik1sUbPn',
  'source': '4beed4b9-b5ad-4d85-af2e-6fa2dcfbed7b',
  'target': '6gGF6bOV',
 

{
    "labels": ["Faculty"],
    "id": 6,
    "properties": {
    "x": "xx",
    "xxx": [],
    }
},


{
    "type": "has_interest",
    "properties": {},
    "id": 147,
    "source": 21,
    "target": 344,
    "value": 1
},